In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchmetrics import MeanSquaredError
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from matplotlib.pyplot import figure
figure(figsize=(18, 12))

In [ ]:
train_data = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
test_data = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
sample_data = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/sample_submission.csv')
print(train_data.shape), print(test_data.shape),print(sample_data.shape)

In [ ]:
train_data.head()

In [ ]:
ax=train_data.groupby('anchor')['id'].count().sort_values(ascending=False).head(10).plot(kind='bar')
for container in ax.containers:
    ax.bar_label(container)
plt.title("top 10 anchor in data")
# plt.xticks(rotation=60)
plt.show()

In [ ]:
ax=train_data.groupby('target')['id'].count().sort_values(ascending=False).head(10).plot(kind='bar')
for container in ax.containers:
    ax.bar_label(container)
plt.title("top 10 target in data")
plt.show()

In [ ]:
ax=train_data.groupby('context')['id'].count().sort_values(ascending=False).head(10).plot(kind='bar')
for container in ax.containers:
    ax.bar_label(container)
plt.title("top 10 context in data")
plt.show()

In [ ]:
train_data.score.value_counts(normalize=True).sort_index().plot(kind='bar', title='Score',xlabel="Level Score")

In [ ]:
train_data.assign(Len_Target=train_data.target.astype(str).str.len())\
      .filter(items=['score','Len_Target'])\
      .groupby('score').Len_Target.sum().plot(kind='bar',title='Score by Len String');

In [ ]:
train_data.assign(Len_Target=train_data.target.astype(str).str.split()\
       .apply(len)).filter(items=['score','Len_Target'])\
       .groupby('score').Len_Target.sum().plot(kind='bar',title='Score by Words');

In [ ]:
def Levenshtein(s0, s1):
        if s0 is None:
            raise TypeError("Argument s0 is NoneType.")
        if s1 is None:
            raise TypeError("Argument s1 is NoneType.")
        if s0 == s1:
            return 0.0
        if len(s0) == 0:
            return len(s1)
        if len(s1) == 0:
            return len(s0)

        v0 = [0] * (len(s1) + 1)
        v1 = [0] * (len(s1) + 1)

        for i in range(len(v0)):
            v0[i] = i

        for i in range(len(s0)):
            v1[0] = i + 1
            for j in range(len(s1)):
                cost = 1
                if s0[i] == s1[j]:
                    cost = 0
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            v0, v1 = v1, v0

        return v0[len(s1)]

def distance(s0,s1):
    if s0 == s1:
            return 0.0
        
    m_len = max(len(s0), len(s1))
    if m_len == 0:
        return 0.0
     
    return Levenshtein(s0, s1) / m_len

def similarity(s0, s1):
        return 1.0 - distance(s0, s1)

In [ ]:
def Leveshtein_Metric(x):
    "aux funct"
    a=similarity(x['anchor'],x['target'])
    return a

In [ ]:
LMetric=train_data.apply(Leveshtein_Metric,axis=1)

In [ ]:
LMetric=LMetric.to_frame('Leveshtein')
LMetric['score']=train_data.score.copy()

In [ ]:
sns.set_style('darkgrid')
sns.relplot(x='Leveshtein',y='score',data=LMetric,height=10);

In [ ]:
sns.histplot(
    LMetric,
    x="Leveshtein", hue="score",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".7",
    linewidth=.7
);

In [ ]:
LMetric.corr()

In [ ]:
LMetric_Test=test_data.apply(Leveshtein_Metric,axis=1)

In [ ]:
sample_data.score = LMetric_Test
sample_data.to_csv("submission.csv", index=False)

In [ ]:
sample_data.head()